In [1]:
# LOAD PACKAGES

import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import rioxarray as rxr
import geopandas as gpd
import xarray as xr
import pandas as pd
import glob
import os
import fnmatch
import math
from math import e
from osgeo import gdal 
print("packages loaded")

packages loaded


In [2]:
# SET DIRECTORIES

# Set basin directory
dnbr_basins = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/output/'
print(sorted(os.listdir(dnbr_basins)))

# Set x_value directory
x_values = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/x_values.csv'
print(x_values)

# Set output directory
out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/basin_pixel_values/'

# Create dataframe 
df = pd.read_csv(x_values, delimiter = ',')
print(df)



['12042_dnbr.tif', '12057_dnbr.tif', '12518_dnbr.tif', '12524_dnbr.tif', '12527_dnbr.tif', '12616_dnbr.tif', '12860_dnbr.tif', '12997_dnbr.tif', '13281_dnbr.tif', '17346_dnbr.tif', '17347_dnbr.tif', '17987_dnbr.tif', '19165_dnbr.tif', '19167_dnbr.tif', '19381_dnbr.tif', '19384_dnbr.tif', '19450_dnbr.tif', '19451_dnbr.tif', '19510_dnbr.tif', '19512_dnbr.tif', '19629_dnbr.tif', '19632_dnbr.tif', '19647_dnbr.tif', '19649_dnbr.tif', '19777_dnbr.tif', '20029_dnbr.tif', '20031_dnbr.tif', '20253_dnbr.tif', '20254_dnbr.tif', '20303_dnbr.tif', '20308_dnbr.tif', '20384_dnbr.tif', '20386_dnbr.tif', '20499_dnbr.tif', '20504_dnbr.tif', '20650_dnbr.tif', '20729_dnbr.tif', '20734_dnbr.tif', '21009_dnbr.tif', '21078_dnbr.tif', '21083_dnbr.tif', '21224_dnbr.tif', '21500_dnbr.tif', '21513_dnbr.tif', '21634_dnbr.tif', '22075_dnbr.tif', '22668_dnbr.tif']
/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/x_values.csv
    basin_id        x1        x2        x3
0      12042  0.197747  0.233633  0.13000

In [15]:
# GRAB PIXEL VALUE FROM EACH BASIN AND OUTPUT TO .CSV
# Coefficients for Southern California
b = -3.63
b_1 = 0.41
b2 = 0.67
b_2 = np.array(b2)
#print(type(b_2))
b_3 = 0.7

#R = 3
#R = 4
#R = 5
R1 = 6.0
R = np.array(R1)
#R = 7
#R = 8
#R = 9
#R = 10

# Iterate over files
for filename in sorted(os.listdir(dnbr_basins)):
    f = os.path.join(dnbr_basins, filename)
    if filename.endswith('19384_dnbr.tif'):
        

        
        # Set empty list for .tif pixel array value
        
        
        #read in tif file
        basin = rxr.open_rasterio(f, masked=True)
        #print(basin)
        
        # Flatten the array, drop nan values, and append array values to list
        bflat = basin.values.flatten()
        #print(bflat)
        pixarr = np.empty((bflat.shape[0]), dtype= float)
        pixarr[:] = np.nan
        #print(pixarr)
        
        x = np.where(np.isnan(bflat) == False)
        print(x)
        
        
        
        # GEtting the data from bflat
        dnbrdata = bflat[x]
        print(dnbrdata)
            
        # Do a bunch of stuff to this data
        #pixarr[x] = newstuff
        
        # Reshape into image
        
        
        #bflatnan = bflat[~np.isnan(bflat)]
        #print(bflatnan)
        #pixarr.extend(bflatnan.tolist())
        #print(pixarr)
        
        # Grab file name for file output
        #b_name = os.path.splitext(filename)[0]
        b_name = filename.split('_')
        b_name_int = int(b_name[0])
        #print(int(b_name[0]))
        individual_basin = df.index[df['basin_id'] == b_name_int]
        
        
        
        x_1 = df['x1'][individual_basin]
        x_3 = df['x3'][individual_basin]
        
        
        s = np.multiply(dnbrdata, b_2)
        s1 = np.multiply(s, R)
        print(s1)
        
        p1 = []
        
        
       
        for i in s1:
            lnx =  b + (b_1 * x_1 * R1) + i + (b_3 * x_3 * R1)
            #print(lnx)
            prob = (e ** lnx) / (1.0 + e ** lnx)
            p1.append(prob)

        #print(p1)
        prob_arr = np.array(p1)
        #print(prob_arr)
        p_final = prob_arr.flatten()
        print(p_final)
        
        pixarr[x] = p_final    
        #z = np.where(np.isnan(pixarr) == False)
        #print(z)
        pixarrImg = pixarr.reshape((basin.shape[1], basin.shape[2]))
        #print(pixarrImg)
        #y = np.where(np.isnan(pixarrImg) == False)
        #print(y)
        
        out_dnbr = xr.Dataset()
        out_dnbr['data'] = xr.DataArray(pixarrImg, dims = ('y', 'x'), coords = {'x': basin.coords['x'], 'y': basin.coords['y']})
        
        test_out = out + str(b_name_int) + '_pv_dnbr'
        out_dnbr.rio.to_raster(test_out, driver='GTIFF')
print('Pixel values generated for basins')

(array([   46,    47,    48, ..., 34057, 34058, 34059]),)
[0.31995744 0.39192712 0.31657484 ... 0.27780354 0.17405224 0.08638594]
[1.2862289  1.5755471  1.2726309  ... 1.1167703  0.69969004 0.34727147]
[0.48070539 0.55282746 0.47731191 ... 0.43864197 0.33989647 0.26577329]
Pixel values generated for basins
